In [2]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import gc

In [3]:
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
file_format = '{}_features_matrixregV4_norm.csv'
day_list = ['day7', 'day8', 'day9']
df_dict = {}
for file in ['day7', 'day8', 'day9','test']: 
    df_dict[file] = pd.read_csv(load_path+file_format.format(file))
    print(file_format.format(file))

day7_features_matrixregV4_norm.csv
day8_features_matrixregV4_norm.csv
day9_features_matrixregV4_norm.csv
test_features_matrixregV4_norm.csv


# Load Ip

In [4]:
# path = '/home/kai/data/kaggle/talkingdata/data/'
# train = pd.read_csv(path + 'train_cleaned_final.csv')
# print('loading training done')
# test = pd.read_csv(path + 'test_cleaned_final.csv')
# print('loading testing done')

# df_hour = pd.read_csv(path+'hourdistri.csv', index_col='Unnamed: 0')
# index = {}
# for day in ['day7', 'day8','day9']:
#     print(day)
#     index[day] = list(range(df_hour.loc[day,'4start'], df_hour.loc[day,'6end0sec'])) + \
#     list(range(df_hour.loc[day,'9start'], df_hour.loc[day,'11end0sec'])) + \
#     list(range(df_hour.loc[day,'13start'], df_hour.loc[day,'15end0sec'])) 
#     df_dict[day]['ip'] = train.loc[index[day], 'ip'].values
# print('processing test!')
# df_dict['test']['ip'] = test['ip']


# just use categorical values that exist in both train and test
no improvement  
ip will not help

In [5]:
# def intersec_category(df_train_all, df_test_all, df, col_list):
#     for col in col_list:
#         print('processing: {}'.format(col))
#         train_index = set(df_train_all[col].value_counts().index)
#         test_index = set(df_test_all[col].value_counts().index)                 
#         inter_index = list(train_index.intersection(test_index))
#         print('train index length is: {}'.format(len(train_index)))
#         print('test index length is: {}'.format(len(test_index)))
#         print('intersection index length is: {}'.format(len(inter_index)))
#         index_map = pd.Series(inter_index, index=inter_index)
#         df.loc[:,col] = df[col].map(index_map).fillna(-1)
#     return df

# col_list = [ 'app', 'device', 'os', 'channel']
# df_train_all = pd.concat([df_dict[day_list[0]],df_dict[day_list[1]], df_dict[day_list[2]]])
# # for file in ['day7', 'day8', 'day9','test']: 
# for file in ['day7']:
#     temp = intersec_category(df_train_all, df_dict['test'], df_dict[file],col_list)

# define parameter

In [6]:
target = 'is_attributed'

day_list = ['day7', 'day8', 'day9']

combine = 0
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 2000,
        'learning_rate': 0.1,
        'num_leaves': 61,
        'num_threads': 4, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 390,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 0.7, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'early_stopping_round':100,
        'bagging_fraction': 0.7, #Randomly select part of data without resampling
        'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        'verbose': 0,
        'scale_pos_weight': 400,
        'metric' : [ 'auc']
    }

categorical_col = [ 'app', 'device', 'os', 'channel', 'hour']
feature_cols = list(df_dict['test'].columns.values)

In [7]:
df_dict['day7'].columns.values

array(['ip_day_hour_count', 'ip_os_day_hour_count',
       'ip_app_day_hour_count', 'ip_app_os_day_hour_count',
       'app_day_hour_count', 'ip_device_os_count',
       'ip_app_device_os_count', 'ip_device_os_mean',
       'ip_app_device_os_mean', 'ip_app_device_mean', 'app_device_os_mean',
       'ip_device_os_time2nextclick', 'ip_app_device_os_time2nextclick',
       'ip_app_device_time2nextclick', 'app_device_os_time2nextclick',
       'ip_device_os_time2previousclick',
       'ip_app_device_os_time2previousclick',
       'ip_app_device_time2previousclick',
       'app_device_os_time2previousclick', 'ip_device_os_countfromfuture',
       'ip_app_device_os_countfromfuture', 'ip_app_device_countfromfuture',
       'app_device_os_countfromfuture', 'ip_device_os_countfrompast',
       'ip_app_device_os_countfrompast', 'ip_app_device_countfrompast',
       'app_device_os_countfrompast', 'ip_device_os_lasttimediff',
       'ip_app_device_os_lasttimediff', 'ip_app_device_lasttimediff',
  

In [8]:
# get rid of extra features
# extra = set(['matrixFact_user_iposdeviceapp_item_channel', 'matrixFact_user_iposdeviceapp_item_device'])
extra = set(['app_device_os_mean',
             'app_device_os_time2nextclick', 
             'app_device_os_time2previousclick',
             'app_device_os_countfromfuture',
             'app_device_os_countfrompast', 
             'app_device_os_lasttimediff', 
             'app_device_os_regression'])
# extra = set(['ip_app_device_mean',
#              'ip_app_device_time2nextclick', 
#              'ip_app_device_time2previousclick',
#              'ip_app_device_countfromfuture',
#              'ip_app_device_countfrompast', 
#              'ip_app_device_lasttimediff', 
#              'ip_app_device_regression'])
feature_cols = list(set(feature_cols) - extra)

In [9]:
feature_cols

['ip_app_device_time2nextclick',
 'ip_device_os_count',
 'ip_app_day_hour_count',
 'ip_device_os_time2nextclick',
 'ip_app_device_os_lasttimediff',
 'ip_app_device_countfrompast',
 'ip_app_device_os_countfrompast',
 'hour',
 'ip_device_os_countfrompast',
 'ip_app_device_os_mean',
 'ip_device_os_mean',
 'ip_app_device_mean',
 'ip_app_device_countfromfuture',
 'channel',
 'ip_app_device_os_time2previousclick',
 'ip_app_device_os_time2nextclick',
 'ip_app_os_day_hour_count',
 'ip_app_device_time2previousclick',
 'ip_device_os_lasttimediff',
 'app_day_hour_count',
 'ip_day_hour_count',
 'ip_app_device_lasttimediff',
 'matrixFact_user_ip_item_appdeviceos',
 'ip_app_device_os_regression',
 'app',
 'ip_app_device_os_count',
 'os',
 'ip_device_os_time2previousclick',
 'ip_app_device_regression',
 'matrixFact_user_iposdeviceapp_item_app',
 'ip_app_device_os_countfromfuture',
 'ip_device_os_countfromfuture',
 'device',
 'ip_os_day_hour_count',
 'ip_device_os_regression']

In [10]:
def train_lightgbm(x_train, x_val, feature_cols, categorical_feature, params, best_round = None, target='is_attributed'):
    param = params.copy()
    y_train = x_train[target].values
    y_val = x_val[target].values
    
    lgb_train = lgb.Dataset(x_train[feature_cols], y_train, categorical_feature = categorical_feature)
    lgb_val = lgb.Dataset(x_val[feature_cols], y_val, categorical_feature = categorical_feature)
    if best_round is not None:
        param['num_rounds'] = best_round
        del param['early_stopping_round']
    print('start training')
    model = lgb.train(param, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)
    return model

In [11]:
for day in day_list:
    train_day = list(set(day_list)-set([day]))
    file_name = 'concat_{}{}_val_{}_predtest_matrixregV4_norm_exclude.npy'.format(train_day[0],train_day[1],day)
    print(file_name)
    trainset = pd.concat([df_dict[train_day[0]],df_dict[train_day[1]]])
    valset = df_dict[day]
    print('building train val done!')
    model = train_lightgbm(trainset, valset, feature_cols, categorical_col, params)
    best_round = model.best_iteration
    df_all = pd.concat([trainset, valset])
    model = train_lightgbm(df_all, valset, feature_cols, categorical_col, params, best_round)
    del df_all
    gc.collect()
    pred = model.predict(df_dict['test'][feature_cols])
    np.save(load_path+file_name, pred)
    
    # prediction
    df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')
    print('loading file done!')
    df_sub = pd.DataFrame()
    df_sub['click_id'] = df_test_raw['click_id']
    df_sub['is_attributed'] = pred
    print('predicting file done!')
    df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/equal_hour_{}{}_val_{}_matrixregV4_norm_exclude.csv.gz'.format(train_day[0],train_day[1],day), compression='gzip', index=False)

    

concat_day9day8_val_day7_predtest_matrixregV4_norm_exclude.npy
building train val done!
start training


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 100 rounds.
[10]	valid_0's auc: 0.979328
[20]	valid_0's auc: 0.981329
[30]	valid_0's auc: 0.982627
[40]	valid_0's auc: 0.983273
[50]	valid_0's auc: 0.983859
[60]	valid_0's auc: 0.984108
[70]	valid_0's auc: 0.984314
[80]	valid_0's auc: 0.984415
[90]	valid_0's auc: 0.984426
[100]	valid_0's auc: 0.984487
[110]	valid_0's auc: 0.984508
[120]	valid_0's auc: 0.984504
[130]	valid_0's auc: 0.984516
[140]	valid_0's auc: 0.984495
[150]	valid_0's auc: 0.984465
[160]	valid_0's auc: 0.984431
[170]	valid_0's auc: 0.984402
[180]	valid_0's auc: 0.98438
[190]	valid_0's auc: 0.984371
[200]	valid_0's auc: 0.984374
[210]	valid_0's auc: 0.984349
[220]	valid_0's auc: 0.984319
Early stopping, best iteration is:
[128]	valid_0's auc: 0.984524
start training
[10]	valid_0's auc: 0.98063
[20]	valid_0's auc: 0.982889
[30]	valid_0's auc: 0.984395
[40]	valid_0's auc: 0.98553
[50]	valid_0's auc: 0.986528
[60]	valid_0's auc: 0.987244
[70]	valid_0's auc: 0.987811
[80]	v

In [36]:
importance = pd.Series(model.feature_importance(), index=feature_cols)
importance = importance.sort_values(ascending=False)
if len(model.feature_importance()) != len(feature_cols):
    raise ValueError('Feature importance has length: {}, \n while feature number is {}'.
                     format(len(model.feature_importance()), len(feature_cols)))
    
importance

channel                                   1663
os                                         783
app                                        596
ip_app_device_os_time2nextclick            578
matrixFact_user_ip_item_appdeviceos        369
matrixFact_user_iposdeviceapp_item_app     331
ip_day_hour_count                          328
ip_device_os_count                         235
ip_app_device_os_count                     209
ip_app_device_os_lasttimediff              208
hour                                       168
ip_device_os_lasttimediff                  149
ip_app_device_time2nextclick               139
ip_app_device_lasttimediff                 138
ip_app_device_time2previousclick           135
ip_app_device_countfrompast                134
ip_device_os_mean                          129
ip_app_device_countfromfuture              121
app_day_hour_count                         115
ip_app_os_day_hour_count                   115
ip_app_device_os_countfromfuture           107
ip_app_device

# Average

In [12]:
#### load each and average
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
day7 = np.load(load_path+'concat_day8day9_val_day7_predtest_matrixregV4_norm_exclude.npy')
day8 = np.load(load_path+'concat_day7day9_val_day8_predtest_matrixregV4_norm_exclude.npy')
day9 = np.load(load_path+'concat_day7day8_val_day9_predtest_matrixregV4_norm_exclude.npy')

pred = (day7 + day8 + day9)/3

In [13]:
# prediction
df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')
print('loading file done!')
df_sub = pd.DataFrame()
df_sub['click_id'] = df_test_raw['click_id']
df_sub['is_attributed'] = pred
print('predicting file done!')
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/equal_hour_blend_matrixregV4_norm_exclude.csv.gz', compression='gzip', index=False)


loading file done!
predicting file done!
